In [1]:
# mean_cookieTheft_picnicScene_intersection.ipynb cell
import os, sys
import pandas as pd

# add project root
sys.path.append("/Users/gilanorup/Desktop/Studium/MSc/MA/code/masters_thesis_gn/src")
from config.constants import GIT_DIRECTORY
from data_preparation.data_cleaning_helpers import impute_mean_dataframe

BASE = os.path.join(GIT_DIRECTORY, "results", "features")
ID = "Subject_ID"

# load your final filtered task files (the exact ones used in analysis)
cookie = pd.read_csv("/Users/gilanorup/Desktop/Studium/MSc/MA/code/masters_thesis_gn/results/features/filtered/cookieTheft_filtered2.csv")
picnic = pd.read_csv("/Users/gilanorup/Desktop/Studium/MSc/MA/code/masters_thesis_gn/results/features/filtered/picnicScene_filtered2.csv")
cookie[ID] = cookie[ID].astype(str)
picnic[ID] = picnic[ID].astype(str)

# same subject exclusions (union)
EXCLUDE_CT = {138, 141, 469, 516, 1065}
EXCLUDE_PS = {390}
EXCLUDE = {str(x) for x in (EXCLUDE_CT | EXCLUDE_PS)}
cookie = cookie[~cookie[ID].isin(EXCLUDE)]
picnic = picnic[~picnic[ID].isin(EXCLUDE)]

# compute intersection of features
feat_ct = set(cookie.columns) - {ID}
feat_ps = set(picnic.columns) - {ID}
FEATURE_INTERSECTION = sorted(feat_ct & feat_ps)

# keep only intersection + ID
cookie = cookie[[ID] + FEATURE_INTERSECTION].copy()
picnic = picnic[[ID] + FEATURE_INTERSECTION].copy()

# inner join subjects present in both (so the mean is well-defined)
m = cookie.merge(picnic, on=ID, suffixes=("_ct", "_ps"), how="inner")

# feature-wise mean
out = pd.DataFrame({ID: m[ID]})
for col in FEATURE_INTERSECTION:
    out[col] = m[[f"{col}_ct", f"{col}_ps"]].mean(axis=1, skipna=True)

# mean-impute any remaining NaNs
out = impute_mean_dataframe(out, exclude_cols=[ID])

# save
out_path = os.path.join(BASE, "filtered", "picture_description_mean_filtered.csv")
os.makedirs(os.path.dirname(out_path), exist_ok=True)
out.to_csv(out_path, index=False)
print("saved →", out_path, "shape:", out.shape)


saved → /Users/gilanorup/Desktop/Studium/MSc/MA/code/masters_thesis_gn/results/features/filtered/picture_description_mean_filtered.csv shape: (995, 54)
